In [1]:
from transformers import AdamWeightDecay
import tensorflow as tf
from tensorflow.keras import mixed_precision
import pandas as pd
import numpy as np
import random
import numpy as np
import nltk

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
# Whether to limit all GPU memory growth True to limit
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu, enable=True)
# GPU uses mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)

from transformers import AutoTokenizer
from transformers import TFT5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq

random.seed(42)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce GTX 1650 Ti, compute capability 7.5
Compute dtype: float16


In [2]:
# Import a T5 model word splitter, which has an embedding function that converts the input text into a T5 model input


# 1. importing already processed data

In [3]:
from datasets import load_dataset, load_metric

metric = load_metric("./rouge_n.py")
train_news_data = load_dataset("csv", data_files="./Data/train_text_summary.csv")
test_news_data = load_dataset("csv", data_files="./Data/test_text_summary.csv")

Using custom data configuration default-9ad319b888548acb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\HU\.cache\huggingface\datasets\csv\default-9ad319b888548acb\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-3a165a301015a263


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\HU\.cache\huggingface\datasets\csv\default-3a165a301015a263\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# 2. pass the tokenizer and modify the data

In [32]:
max_input_length = 10000
max_target_length = 10000
prefix = "summarize: "
def preprocess_function(removed_news_data):

    inputs = [prefix+doc for doc in removed_news_data["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            removed_news_data["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [33]:
train_tokenized_datasets = train_news_data.map(preprocess_function, batched=True)
# test_tokenized_datasets = test_news_data.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\HU\.cache\huggingface\datasets\csv\default-5433116e5e2e416b\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58\cache-1c80317fa3b1799d.arrow


In [37]:
# Import T5 models
model = TFT5ForConditionalGeneration.from_pretrained("./t5-small")
# Use transformer-specific data sorting to build data pipelines
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# 3. build the data pipeline

In [40]:
# Define the hyperparameters and the data pipeline
# For now, we only consider the training set, so there is only one train below
# If we use test, we need to write one more test to handle code_cell, 
# because test_text_summary has some mixed data, and only pure str data can pass the tokenizer of T5.
batch_size = 1
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

train_dataset = train_tokenized_datasets["train"].to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
# test_dataset = test_tokenized_datasets["train"].to_tf_dataset(
#     batch_size=batch_size,
#     columns=["input_ids", "attention_mask", "labels"],
#     shuffle=False,
#     collate_fn=data_collator,
# )

In [41]:
# Define AUTOTUNE so that data in memory can be multi-threaded into the GPU
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
# train_dataset = dtrain_datasets.cache().shuffle(2000).prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# 4. define the ROUGE series of evaluation functions

In [43]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

# 5. Define the optimizer and training process of the model (if the video memory is sufficient, the model can be trained)

In [44]:
# Define the optimizer for the model and compile the model
# Note that the T5 model as transformer generally has a built-in loss function that we don't have to redesign
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [281]:
# Be careful not to run this code, it is likely to be unable to train because of insufficient video memory, 
# estimated video memory needs about 12G to run smoothly
# Train the model and save the logs to the logs for viewing with tensorboard
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=test_dataset, predict_with_generate=True
)

callbacks = [metric_callback, tensorboard_callback]

model.fit(train_dataset, validation_data=None, epochs=1, callbacks=callbacks)
# model.fit(train_dataset, validation_data = test_dataset, epochs=1, batch_size=1, callbacks=callbacks)

  20/1381 [..............................] - ETA: 10:48 - loss: nan

ResourceExhaustedError: Graph execution error:

Detected at node 'tft5_for_conditional_generation_1/decoder/block_._4/layer_._1/EncDecAttention/einsum/Einsum' defined at (most recent call last):
    File "D:\Anaconda3\envs\hp\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "D:\Anaconda3\envs\hp\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "D:\Anaconda3\envs\hp\Lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\Anaconda3\envs\hp\Lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "D:\Anaconda3\envs\hp\Lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "D:\Anaconda3\envs\hp\Lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "D:\Anaconda3\envs\hp\lib\asyncio\base_events.py", line 534, in run_forever
   

# 6. inference part, test model effect

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./t5-small")
from transformers import TFT5ForConditionalGeneration
model = TFT5ForConditionalGeneration.from_pretrained("./t5-small")
import pandas as pd

def out_summary(text):
    ip = tokenizer(text, max_length=None, truncation=True, return_tensors='tf')

    summary_ids = model.generate(ip['input_ids'],max_length=600,min_length=None)
    print(ip)

    print('\nsummary_ids = ', summary_ids)

    # print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])
    summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    # print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False))
    return summary

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at ./t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
text = """
        summarize: (CNN)For the second time during his papacy, Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.
        Pope Francis said Sunday that he would hold a meeting of cardinals on February 14 "during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world," according to Vatican Radio.
        New cardinals are always important because they set the tone in the church and also elect the next pope, CNN Senior Vatican Analyst John L. Allen said. They are sometimes referred to as the princes of the Catholic Church.
        The new cardinals come from countries such as Ethiopia, New Zealand and Myanmar.
        "This is a pope who very much wants to reach out to people on the margins, and you clearly see that in this set," Allen said. "You're talking about cardinals from typically overlooked places, like Cape Verde, the Pacific island of Tonga, Panama, Thailand, Uruguay."
        But for the second time since Francis' election, no Americans made the list.
        "Francis' pattern is very clear: He wants to go to the geographical peripheries rather than places that are already top-heavy with cardinals," Allen said.
        Christopher Bellitto, a professor of church history at Kean University in New Jersey, noted that Francis announced his new slate of cardinals on the Catholic Feast of the Epiphany, which commemorates the visit of the Magi to Jesus' birthplace in Bethlehem.
        "On feast of three wise men from far away, the Pope's choices for cardinal say that every local church deserves a place at the big table."
        In other words, Francis wants a more decentralized church and wants to hear reform ideas from small communities that sit far from Catholicism's power centers, Bellitto said.
        That doesn't mean Francis is the first pontiff to appoint cardinals from the developing world, though. Beginning in the 1920s, an increasing number of Latin American churchmen were named cardinals, and in the 1960s, St. John XXIII, whom Francis canonized last year, appointed the first cardinals from Japan, the Philippines and Africa.
        In addition to the 15 new cardinals Francis named on Sunday, five retired archbishops and bishops will also be honored as cardinals.
        Last year, Pope Francis appointed 19 new cardinals, including bishops from Haiti and Burkina Faso.
        CNN's Daniel Burke and Christabelle Fombu contributed to this report.
"""

In [13]:
text = '''summarize: Us officials are losing confidence that even if they work with Allies to provide Ukraine with heavier and more advanced weapons. the country will not be able to achieve its insistent goal of retaking all the territory seized by Russia in the past four months. CNN reported it on Sunday.'''

In [14]:
print(text)

summarize: Us officials are losing confidence that even if they work with Allies to provide Ukraine with heavier and more advanced weapons. the country will not be able to achieve its insistent goal of retaking all the territory seized by Russia in the past four months. CNN reported it on Sunday.


In [15]:
out_summary(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': <tf.Tensor: shape=(1, 63), dtype=int32, numpy=
array([[21603,    10,  6952,  4298,    33,  5489,  3410,    24,   237,
            3,    99,    79,   161,    28,   432,   725,    12,   370,
        11897,    28, 21648,    11,    72,  2496,  7749,     5,     8,
          684,    56,    59,    36,     3,   179,    12,  1984,   165,
        10419,   295,  1288,    13,     3,    60, 14867,    66,     8,
         9964,     3, 27217,    57,  4623,    16,     8,   657,   662,
          767,     5, 19602,  2196,    34,    30,  1771,     5,     1]])>, 'attention_mask': <tf.Tensor: shape=(1, 63), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

summary_ids =  tf.Tensor(
[[    0   178  4298    33  5489  3410    24   237     3    99    79   161
     28   432   725    12   370 11897    28 21648    11

['us officials are losing confidence that even if they work with Allies to provide Ukraine with heavier and more advanced weapons . the country will not be able to achieve its insistent goal of retaking all the territory seized by Russia in the past four months .']

# ---------------------------------------------------

# 7. ROUGE and BLEU score in test

In [8]:
# load the data form csv
test_text_summary = pd.read_csv("./Data/test_text_summary.csv", low_memory=False)
test_texts = test_text_summary["text"]
test_summaries = test_text_summary["summary"]

In [22]:
# get the summaries form test texts
predict_summaries_test = []
for i in range(len(test_texts[:100])):
    summary = out_summary(test_texts[i])
    predict_summaries_test.append(summary)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [33]:
predict_summaries_test

[['a $37 "art craft/toy" smuggled into the U.S. in 2001 . the painting was shipped to the port of Newark and was seized . the painting was smuggled into the u.s. in a package that described it as a $37 "art craft" . the painting was smuggled into the u.s. in 2001 .'],
 [': the Obama campaign has received at least $1,000 to Romney’s campaign . NEWLINE_CHAR NEWLINE_CHAR The Obama campaign has received at least $1,000 . NEWLINE_CHAR NEWLINE_CHAR The Obama campaign has received at least $1,000 .'],
 ['NEWLINE_CHAR NEWLINE_CHAR WASHINGTON—Behind the political showdown over the deadly U.S. consulate attack in Benghazi, Libya, are a few dozen heavily edited words . NEWLINE_CHAR NEWLINE_CHAR The 94-word intelligence summary emerged from a daylong email debate between more than two dozen intelligence officials . that moment has become a proxy for a ... Article Excerpt NEWLINE_CHAR NEWLINE_CHAR WASHINGTON—Behin'],
 ['NEWLINE_CHAR NEWLINE_CHAR NEWLINE_CHAR TODAY . the couple are suing the product

In [36]:
# store the predict data as csv
T5_predict = []
for i in range(len(predict_summaries_test)):
    T5_predict.append(predict_summaries_test[i][0])
    
DF = {"reference":test_summaries[:100], "T5_predict":T5_predict}
df = pd.DataFrame(DF)
df.to_csv('./Data/T5_predict_summaries.csv', index=False)

In [30]:
predict_summaries_test[1][0]

': the Obama campaign has received at least $1,000 to Romney’s campaign . NEWLINE_CHAR NEWLINE_CHAR The Obama campaign has received at least $1,000 . NEWLINE_CHAR NEWLINE_CHAR The Obama campaign has received at least $1,000 .'

In [29]:
test_texts[1]

'High-profile Hollywooders, as well as several sports, business and political luminaries, are offering up a string of last-minute, big-dollar contributions both Barack Obama and Mitt Romney. NEWLINE_CHAR NEWLINE_CHAR Actors Jake Gyllenhaal, Zach Galifianakis, Renee Zellweger, Susan Saint James Ebersol and Amanda Seyfried are among the four-figure donors in recent days to Obama’s campaign, according to federal campaign finance disclosures. NEWLINE_CHAR NEWLINE_CHAR Text Size - NEWLINE_CHAR NEWLINE_CHAR + NEWLINE_CHAR NEWLINE_CHAR reset PHOTOS: Stars hit up the swing states Play Slideshow POLITICO’s late night roundup NEWLINE_CHAR NEWLINE_CHAR Author Anne Rice, Disney TV Animation Executive Producer Dan Povenmire, and comedy writer Ian Maxtone-Graham are other notable Obama contributors. NEWLINE_CHAR NEWLINE_CHAR For Romney, actor Kelsey Grammer, Denver Broncos executive John Elway and former game show host Bob Barker have made 11th hour, four-figure donations. NEWLINE_CHAR NEWLINE_CHAR 

In [27]:
test_summaries[:100][1]

'e stars are coming out in a big way as Election Day looms. The latest: Will Ferrell really, really wants you to vote—and he\'ll do anything to make sure you do, he promises in a new video for President Obama. "Hungry? How about a home-cooked meal? Hope you like angel hair pasta," he says. "You need a guy to help you move a couch? Done. I\'ve even got my own van." But his promises get bigger—and weirder—from there. "If you vote, I\'ll eat anything you tell me to—garbage, hair, human toenails, underpants, whatever—I\'ll do it," he swears. The president\'s name stays out of it until the very end, when Ferrell concludes, while holding a football, "Vote Obama. It\'s a slam dunk." On Jimmy Kimmel\'s show last week, Chris Rock offered up a special message to white voters. "In times like these, you need a white president you can trust," Rock explains. "And that white president\'s name is Barack Obama." His evidence that Obama is actually white? The president used to be called "Barry," he like

In [32]:
# Import rouge Blue and calculate score, get Rouge-1 and Rouge-2 scores for the first 100 test data
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

def compute_score(predict_summaries_test, test_summaries):
    rouge = Rouge()
    Rouge_1 = []
    Rouge_2 = []
    B = []

    for i in range(len(test_texts[:100])):
        score = rouge.get_scores(predict_summaries_test[i][0], test_summaries[:100][i])
        score_B = sentence_bleu([test_summaries[:100][i]], predict_summaries_test[i])
        B.append(score_B)
        Rouge_1.append(score[0]["rouge-1"]["r"])
        Rouge_2.append(score[0]["rouge-2"]["r"])

    print("Rouge-1 score: ", sum(Rouge_1)/len(Rouge_1))
    print("Rouge-2 score: ", sum(Rouge_2)/len(Rouge_2))
    print("BLEU score: ", sum(B)/len(B))
    
compute_score(predict_summaries_test, test_summaries)

Rouge-1 score:  0.11049193461168452
Rouge-2 score:  0.029912964282240554
BLEU score:  0.0
